In [1]:
import dsgp4

In [2]:
import torch

In [ ]:
# import matplotlib 

In [6]:
tles=dsgp4.tle.load('tle-2025-06-22--2025-06-23.txt')

#tle file processing

filter: recurring satellites, only LEOs
 

In [9]:
import pandas as pd

In [10]:
df = pd.DataFrame(tles)

In [11]:
structured = [tle.__dict__ for tle in tles]

In [12]:
structured = [{
    "satellite_catalog_number": tle.satellite_catalog_number,
    "epoch_year": tle.epoch_year,
    "epoch_days": tle.epoch_days,
    "mean_motion": tle.mean_motion,
    "eccentricity": tle.eccentricity,
    "inclination": tle.inclination,
    "raan": tle.raan,
    "argument_of_perigee": tle.argument_of_perigee,
    "mean_anomaly": tle.mean_anomaly,
    "b_star": tle.b_star,
    
} for tle in tles]

In [13]:
df = pd.DataFrame(structured)

In [14]:
df.head()

,satellite_catalog_number,epoch_year,epoch_days,mean_motion,eccentricity,inclination,raan,argument_of_perigee,mean_anomaly,b_star
0,5,2025,172.535907,0.000790,0.184175,0.597938,6.064593,5.452229,0.583881,-7.583000e-07
1,5,2025,173.547997,0.000790,0.184171,0.597924,6.010083,5.531847,0.524601,-1.066900e-05
2,11,2025,172.912052,0.000865,0.144860,0.573690,0.140982,2.924249,3.430024,5.914600e-04
3,11,2025,173.583211,0.000865,0.144862,0.573697,0.096688,2.990859,3.342299,5.719200e-04
4,12,2025,172.737385,0.000835,0.164851,0.574245,0.959432,6.232141,0.036832,2.593400e-04


In [15]:
df_latest = df.sort_values('epoch_days').groupby('satellite_catalog_number', as_index=False).last()


In [16]:
df_latest

,satellite_catalog_number,epoch_year,epoch_days,mean_motion,eccentricity,inclination,raan,argument_of_perigee,mean_anomaly,b_star
0,5,2025,173.547997,0.000790,0.184171,0.597924,6.010083,5.531847,0.524601,-0.000011
1,11,2025,173.583211,0.000865,0.144862,0.573697,0.096688,2.990859,3.342299,0.000572
2,12,2025,173.085574,0.000835,0.164849,0.574241,0.938043,6.264289,0.014094,0.000262
3,16,2025,173.524451,0.000763,0.202305,0.597986,4.628821,0.728281,5.796460,-0.000063
4,20,2025,173.231396,0.000844,0.164300,0.582100,2.127748,4.978519,0.999833,0.000200
...,...,...,...,...,...,...,...,...,...,...
23726,270376,2025,173.538980,0.000954,0.006668,1.800747,4.627887,4.363089,1.909467,0.018820
23727,270413,2025,173.305646,0.001068,0.014851,1.548990,5.274982,0.130709,6.158382,0.000213
23728,270425,2025,173.298274,0.001064,0.013744,1.544341,5.102067,5.971319,0.305590,0.000594
23729,270429,2025,173.507959,0.001042,0.000703,1.533383,4.727793,5.217318,1.066724,0.001065


In [18]:
with open("leo_ids.txt") as f:
    leo_ids = [int(line.strip()) for line in f]

In [19]:
df_leos = df_latest[df_latest['satellite_catalog_number'].isin(leo_ids)]

In [20]:
df_leos

,satellite_catalog_number,epoch_year,epoch_days,mean_motion,eccentricity,inclination,raan,argument_of_perigee,mean_anomaly,b_star
1,11,2025,173.583211,0.000865,0.144862,0.573697,0.096688,2.990859,3.342299,0.000572
4,20,2025,173.231396,0.000844,0.164300,0.582100,2.127748,4.978519,0.999833,0.000200
5,22,2025,173.569623,0.001105,0.009175,0.877457,1.133652,2.181377,4.118586,0.000342
6,29,2025,173.511843,0.001075,0.002231,0.844373,5.114209,2.819735,3.466465,0.000082
7,45,2025,173.443893,0.001044,0.024300,1.164007,4.039969,1.062652,5.264499,0.000056
...,...,...,...,...,...,...,...,...,...,...
23522,64336,2025,173.583356,0.001147,0.000140,0.750500,2.764403,4.843233,0.142265,-0.006848
23523,64337,2025,173.583356,0.001147,0.000135,0.750504,2.764572,4.746004,0.167806,-0.006813
23524,64338,2025,173.583356,0.001147,0.000147,0.750500,2.764670,4.831646,0.045366,-0.006709
23525,64339,2025,173.583356,0.001147,0.000177,0.750514,2.764481,4.771023,0.178793,-0.006999


In [21]:
#set TLEs, The Near Equatorial Orbit (NEqO) Constellation: teleos 1, teleos 2 , ds-eo, ds-sar neu-sar(decayed)
neqo = [41169, 56310, 52935 , 57481 ]


In [22]:
df_leos[df_leos['satellite_catalog_number'].isin(neqo)] #only teleos2 and ds-eo is found

,satellite_catalog_number,epoch_year,epoch_days,mean_motion,eccentricity,inclination,raan,argument_of_perigee,mean_anomaly,b_star
14596,52935,2025,173.625498,0.001098,0.000302,0.174468,6.034912,2.111211,4.172868,0.000176
16923,56310,2025,173.639124,0.001088,0.000078,0.174350,5.273284,4.707633,1.575769,0.000318


In [23]:
df_neqo = df_leos[df_leos['satellite_catalog_number'].isin(neqo)]
df_sampled = df_leos[~df_leos['satellite_catalog_number'].isin(neqo)].sample(n=100)
df_batch = pd.concat([df_neqo, df_sampled])


In [24]:
df_batch

,satellite_catalog_number,epoch_year,epoch_days,mean_motion,eccentricity,inclination,raan,argument_of_perigee,mean_anomaly,b_star
14596,52935,2025,173.625498,0.001098,0.000302,0.174468,6.034912,2.111211,4.172868,0.000176
16923,56310,2025,173.639124,0.001088,0.000078,0.174350,5.273284,4.707633,1.575769,0.000318
4029,19790,2025,173.062431,0.000920,0.001867,1.441942,3.433747,5.727462,1.040588,0.000174
1450,7768,2025,173.560244,0.000999,0.002441,1.447993,0.616161,1.485361,5.102728,0.000026
20129,60394,2025,173.573841,0.000982,0.001420,1.553038,5.537184,0.015258,6.269977,0.000081
...,...,...,...,...,...,...,...,...,...,...
1378,7297,2025,173.620979,0.001003,0.004498,1.133551,2.433177,1.661761,1.349304,0.000018
21883,62515,2025,173.145858,0.001142,0.000157,0.750485,3.508700,4.382372,1.902030,0.001108
19705,59856,2025,173.073166,0.001111,0.000105,0.750513,5.336667,4.782422,1.502039,-0.000216
17714,57389,2025,173.201107,0.001095,0.000555,1.736295,1.811462,3.232060,3.053159,0.000348


In [25]:
from datetime import datetime, timedelta
import numpy as np

In [26]:
def tle_epoch_to_datetime(row):
    year = int(row['epoch_year'])  # just use as is!
    return datetime(year, 1, 1) + timedelta(days=float(row['epoch_days']))

In [27]:
df_batch['epoch_datetime'] = df_batch.apply(tle_epoch_to_datetime, axis=1)

In [28]:
t_start = min(df_batch['epoch_datetime'])
t_end = t_start + timedelta(days=3)  

step = timedelta(minutes=1)
time_grid = [t_start + i * step for i in range(int((t_end - t_start) / step))]

In [29]:
time_grid[:10]

[Timestamp('2025-06-22 15:44:09.470688'),
 Timestamp('2025-06-22 15:45:09.470688'),
 Timestamp('2025-06-22 15:46:09.470688'),
 Timestamp('2025-06-22 15:47:09.470688'),
 Timestamp('2025-06-22 15:48:09.470688'),
 Timestamp('2025-06-22 15:49:09.470688'),
 Timestamp('2025-06-22 15:50:09.470688'),
 Timestamp('2025-06-22 15:51:09.470688'),
 Timestamp('2025-06-22 15:52:09.470688'),
 Timestamp('2025-06-22 15:53:09.470688')]

In [30]:
epochs = df_batch['epoch_datetime'].tolist()  # length N
tsince_matrix = torch.tensor([
    [(t - epoch).total_seconds() / 60 for epoch in epochs]
    for t in time_grid  # length T
])

In [31]:
tsince_matrix.shape

torch.Size([4320, 102])

In [32]:
#workaround because I deleted some uneccessary fields ealier which i shouldn't
if 'classification' not in df_batch.columns:
    df_batch['classification'] = 'U'

df_batch['international_designator'] = '00000A'
df_batch['mean_motion_first_derivative'] = 0.0
df_batch['mean_motion_second_derivative'] = 0.0
df_batch['revolution_number_at_epoch'] = 0
df_batch['ephemeris_type'] = 0
df_batch['element_number'] = 0

In [33]:
df_batch

,satellite_catalog_number,epoch_year,epoch_days,mean_motion,eccentricity,inclination,raan,argument_of_perigee,mean_anomaly,b_star,epoch_datetime,classification,international_designator,mean_motion_first_derivative,mean_motion_second_derivative,revolution_number_at_epoch,ephemeris_type,element_number
14596,52935,2025,173.625498,0.001098,0.000302,0.174468,6.034912,2.111211,4.172868,0.000176,2025-06-23 15:00:42.995232,U,00000A,0.0,0.0,0,0,0
16923,56310,2025,173.639124,0.001088,0.000078,0.174350,5.273284,4.707633,1.575769,0.000318,2025-06-23 15:20:20.278176,U,00000A,0.0,0.0,0,0,0
4029,19790,2025,173.062431,0.000920,0.001867,1.441942,3.433747,5.727462,1.040588,0.000174,2025-06-23 01:29:54.007296,U,00000A,0.0,0.0,0,0,0
1450,7768,2025,173.560244,0.000999,0.002441,1.447993,0.616161,1.485361,5.102728,0.000026,2025-06-23 13:26:45.098880,U,00000A,0.0,0.0,0,0,0
20129,60394,2025,173.573841,0.000982,0.001420,1.553038,5.537184,0.015258,6.269977,0.000081,2025-06-23 13:46:19.870176,U,00000A,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378,7297,2025,173.620979,0.001003,0.004498,1.133551,2.433177,1.661761,1.349304,0.000018,2025-06-23 14:54:12.558816,U,00000A,0.0,0.0,0,0,0
21883,62515,2025,173.145858,0.001142,0.000157,0.750485,3.508700,4.382372,1.902030,0.001108,2025-06-23 03:30:02.101824,U,00000A,0.0,0.0,0,0,0
19705,59856,2025,173.073166,0.001111,0.000105,0.750513,5.336667,4.782422,1.502039,-0.000216,2025-06-23 01:45:21.561408,U,00000A,0.0,0.0,0,0,0
17714,57389,2025,173.201107,0.001095,0.000555,1.736295,1.811462,3.232060,3.053159,0.000348,2025-06-23 04:49:35.675040,U,00000A,0.0,0.0,0,0,0


In [34]:
tle_dicts = df_batch.to_dict(orient='records')
tles = [dsgp4.tle.TLE(d) for d in tle_dicts]
_, tle_batch = dsgp4.initialize_tle(tles)


In [36]:
tsince_flat = tsince_matrix.flatten()

In [37]:
tsince_len = tsince_flat.shape[0]

In [38]:
tles_ = [tle for tle in tles for _ in range(tsince_len)]

In [183]:


# Initialize TLE batch
_, tle_batch = dsgp4.initialize_tle(tles_)

# Propagate the batch for one day
states_teme = dsgp4.propagate_batch(tle_batch, tsinces)


MemoryError: 